<a href="https://colab.research.google.com/github/joexu22/llama2-finetune/blob/main/Notebooks/Inference_Llama2_7b_hf_ultimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Load model directly from HuggingFace

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

In [ ]:
# Pull Data From Repo
# Hmm... could just use git clone, D'OH
import requests

def download_github_file(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

# Get File (make sure it's raw):
file_url = 'https://raw.githubusercontent.com/joexu22/llama2-finetune/main/data_processing/questions.txt'
save_path = '/content/questions.txt'
download_github_file(file_url, save_path)

In [ ]:
# utilize pulled model
# load model into GPU

import torch

device = "cuda:0"
model = model.to(device)

# extract out all the lines from the list of interview questions
# process it with "llama response generator" getting an interview response
# then write it out to output

with open('/content/questions.txt', encoding='utf-8') as f:
    questions = f.readlines()
    questions = [question.strip() for question in questions if question.strip()]

# write out to file, the generated responses
with open('/content/output.txt', 'w', encoding='utf-8') as f:
    for question in questions:
        # write question to output file
        f.write(question + '\n')

        # Generate the assistant's response
        text = f"### Human: {question} ### Assistant: Hey bud,"
        inputs = tokenizer(text, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=1000)

        # Decode the generated text and write it to the output file
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_text = generated_text.encode('utf-8')
        f.write(str(generated_text) + '\n')